<a href="https://colab.research.google.com/github/arnavjain2710/IITISoC24-ML-36/blob/main/Fine_tuning_the_wav2vec2_lg_xlsr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To load: if the files section isn't empty
First rename tess toronto---- ko underscore se seperate karke rename kardo
TESS Toronto wale ko bhi underscore se seperate karke rename karna hai after u run the first cell, jo upload hoga usme underscore nhi hoga because
create a folder called 'ravdess' in the content section

Firstly loading the datasets from kaggle, as these are too huge to be stored on computer

In [ ]:
! rm -rf ALL
! rm -rf AudioWAV
! rm -rf ravdess
! rm -rf audio_speech_actors_01-24
! rm -rf TESS_Toronto_emotional_speech_set_data
! rm -rf tess_toronto_emotional_speech_set_data
# name all folders as it has to be run before each iteration

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaidehibhat","key":"21ccd7dc13a7f52c2ecc8b29ff2e7dd3"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!mv kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! pip install -q kaggle

In [ ]:
!kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
shreyanshverma27/online-sales-dataset-popular-marketplace-data  Online Sales Dataset - Popular Marketplace Data      7KB  2024-05-25 23:55:26           3084         57  1.0              
devarajv88/walmart-sales-dataset                                e-Commerce (Walmart) Sales Dataset                   5MB  2024-05-30 12:09:55           1988         32  1.0              
ayushparwal2026/country-population-from-1960-to-2022            Country Population from 1960 to 2022                75KB  2024-05-31 08:58:26           1499         39  0.9411765        
mayankanand2701/apple-stock-price-dataset                       A

Loading Crema-D from kaggle, already only audio files are there in the dataset

In [ ]:
! kaggle datasets download -d ejlok1/cremad

Dataset URL: https://www.kaggle.com/datasets/ejlok1/cremad
License(s): ODC Attribution License (ODC-By)
 98% 441M/451M [00:07<00:00, 26.9MB/s]
100% 451M/451M [00:07<00:00, 60.2MB/s]


Loading TESS dataset

In [ ]:
! kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess

Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100% 427M/428M [00:06<00:00, 130MB/s]
100% 428M/428M [00:06<00:00, 72.1MB/s]


Loading SAVEE dataset, has only audio files in it

In [ ]:
! kaggle datasets download -d ejlok1/surrey-audiovisual-expressed-emotion-savee

Dataset URL: https://www.kaggle.com/datasets/ejlok1/surrey-audiovisual-expressed-emotion-savee
License(s): copyright-authors
 93% 100M/107M [00:00<00:00, 136MB/s] 
100% 107M/107M [00:00<00:00, 125MB/s]


Loading RAVDESS dataset, has only audio files in it

In [ ]:
! kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 99% 426M/429M [00:03<00:00, 144MB/s]
100% 429M/429M [00:03<00:00, 135MB/s]


Unzipping the content

In [ ]:
! unzip /content/cremad.zip
! unzip /content/ravdess-emotional-speech-audio.zip
! unzip /content/surrey-audiovisual-expressed-emotion-savee.zip
! unzip /content/toronto-emotional-speech-set-tess.zip

Streaming output truncated to the last 5000 lines.
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_back_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bar_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_base_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bean_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_beg_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bite_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_boat_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bone_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_book_angry.wav  
  inflating: TESS Toronto emotional speech set data/OAF_angry/OAF_bought_angry.wav  
  inflating: TESS Toronto emotional 

Making sense of data, all the datasets have these emotions in common:
1) Anger
2) Disgust
3) Happy
4) Neutral
5) Sad
6) Fear

Tip: remember that RAVDESS labels neutral as calm

Only Crema-D doesn't have surprise hence not including right now, later we can include that too, also when we get access to LSSED database we can start working with that

Work Plan: Create a double numpy array, with indexes as given above (-1 ofc) we will store the file paths in it.

In [ ]:
dataset_array = [[] for _ in range(6)]
#this will contain five arrays on indexes:  0: anger, 1:disgust, 2: Happy, 3: neutral/calm , 4:sad, 5: fear
#further each will contain subarrays containing file paths
#have to create directories of each dataset for it


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
tess_path= '/content/TESS_Toronto_emotional_speech_set_data'
savee_path='/content/ALL'
cremad_path='/content/AudioWAV'
# have to work exrtra for ravdess


In [ ]:
for i in range(1,25):
  if i<10:
    file_path= f'/content/Actor_0{i}'
    print(file_path)
    !mv {file_path} /content/ravdess


  else:
    file_path= f'/content/Actor_{i}'
    print(file_path)
    !mv {file_path} /content/ravdess





/content/Actor_01
/content/Actor_02
/content/Actor_03
/content/Actor_04
/content/Actor_05
/content/Actor_06
/content/Actor_07
/content/Actor_08
/content/Actor_09
/content/Actor_10
/content/Actor_11
/content/Actor_12
/content/Actor_13
/content/Actor_14
/content/Actor_15
/content/Actor_16
/content/Actor_17
/content/Actor_18
/content/Actor_19
/content/Actor_20
/content/Actor_21
/content/Actor_22
/content/Actor_23
/content/Actor_24


In [ ]:
ravdess_path='/content/ravdess'

In [ ]:
#creating directories
ravdess_dirlist=os.listdir(ravdess_path)
savee_dirlist=os.listdir(savee_path)
cremad_dirlist=os.listdir(cremad_path)
tess_dirlist=os.listdir(tess_path)


Converting and labelling the data for RAVDESS

In [ ]:
for dir in ravdess_dirlist:
  actor=os.listdir(ravdess_path+'/' + dir)
  for file in actor:
    split1=file.split('.')[0]
    split2=split1.split('-')
    if split2[2]=='01': #neutral
      dataset_array[3].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='02': #calm
      dataset_array[3].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='03' : #happy
      dataset_array[2].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='04': #sad
      dataset_array[4].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='05' : #angry
      dataset_array[0].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='06': #fear
      dataset_array[5].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='07': #disgust
      dataset_array[1].append(ravdess_path +'/'+ dir + '/' + file)
    elif split2[2]=='08': #suprise, not considered rn
      continue
    else:
      print("Error")

Converting and Labelling data for TESS

In [ ]:
for dir in tess_dirlist:
  folder=os.listdir(tess_path+'/' + dir)
  split=dir.split('_')[1]
  for file in folder:
    if split== 'neutral' or split=='Neutral': #neutral
      dataset_array[3].append(tess_path +'/'+ dir + '/' + file)

    elif split=='happy' or split=='Happy' : #happy
      dataset_array[2].append(tess_path +'/'+ dir + '/' + file)
    elif split=='sad' or split=='Sad': #sad
      dataset_array[4].append(tess_path +'/'+ dir + '/' + file)
    elif split=='angry' or split=='Angry': #angry
      dataset_array[0].append(tess_path +'/'+ dir + '/' + file)
    elif split=='fear'or split=='Fear': #fear
      dataset_array[5].append(tess_path +'/'+ dir + '/' + file)
    elif split=='disgust' or split=='Disgust': #disgust
      dataset_array[1].append(tess_path +'/'+ dir + '/' + file)
    elif split=='pleasant' or split=='Pleasant': #suprise, not considered rn
      continue
    else:
      print("Error")




Converting and Labelling data for SAVEE

In [ ]:
for file in savee_dirlist:
  splits1=file.split('.')[0]
  splits2=splits1.split('_')[1]
  char_array=list(splits2)
  if char_array[0]== 'n': #neutral
    dataset_array[3].append(savee_path + '/' + file)

  elif char_array[0]=='h' : #happy
    dataset_array[2].append(savee_path + '/' + file)
  elif char_array[0]=='s':
    if char_array[1]=='a':
      dataset_array[4].append(savee_path + '/' + file) #sad

    else:
      continue
  elif char_array[0]=='a': #angry
    dataset_array[0].append(savee_path +  '/' + file)
  elif char_array[0]=='f': #fear
    dataset_array[5].append(savee_path +  '/' + file)
  elif char_array[0]=='d' : #disgust
    dataset_array[1].append(savee_path +  '/' + file)

  else:
    print("Error")

Converting and labelling data for Crema-D

In [ ]:
for file in cremad_dirlist:
  splitsee1=file.split('.')[0]
  splitsee2=splitsee1.split('_')[2]
  if splitsee2== 'NEU': #neutral
    dataset_array[3].append(cremad_path + '/' + file)

  elif splitsee2=='HAP' : #happy
    dataset_array[2].append(cremad_path +  '/' + file)
  elif splitsee2=='SAD' : #sad
    dataset_array[4].append(cremad_path +  '/' + file)
  elif splitsee2=='ANG': #angry
    dataset_array[0].append(cremad_path + '/' + file)
  elif splitsee2=='FEA': #fear
    dataset_array[5].append(cremad_path +  '/' + file)
  elif splitsee2=='DIS': #disgust
    dataset_array[1].append(cremad_path + '/' + file)
  else:
    print("Error")

In [ ]:
print(dataset_array[0])

['/content/ravdess/Actor_22/03-01-05-01-02-02-22.wav', '/content/ravdess/Actor_22/03-01-05-01-01-02-22.wav', '/content/ravdess/Actor_22/03-01-05-01-02-01-22.wav', '/content/ravdess/Actor_22/03-01-05-02-02-01-22.wav', '/content/ravdess/Actor_22/03-01-05-02-01-02-22.wav', '/content/ravdess/Actor_22/03-01-05-02-02-02-22.wav', '/content/ravdess/Actor_22/03-01-05-02-01-01-22.wav', '/content/ravdess/Actor_22/03-01-05-01-01-01-22.wav', '/content/ravdess/Actor_02/03-01-05-01-02-01-02.wav', '/content/ravdess/Actor_02/03-01-05-01-01-02-02.wav', '/content/ravdess/Actor_02/03-01-05-01-02-02-02.wav', '/content/ravdess/Actor_02/03-01-05-02-01-02-02.wav', '/content/ravdess/Actor_02/03-01-05-01-01-01-02.wav', '/content/ravdess/Actor_02/03-01-05-02-02-02-02.wav', '/content/ravdess/Actor_02/03-01-05-02-01-01-02.wav', '/content/ravdess/Actor_02/03-01-05-02-02-01-02.wav', '/content/ravdess/Actor_11/03-01-05-02-02-01-11.wav', '/content/ravdess/Actor_11/03-01-05-01-02-02-11.wav', '/content/ravdess/Actor_11/

In [ ]:
print(dataset_array[1])

['/content/ravdess/Actor_22/03-01-07-02-01-02-22.wav', '/content/ravdess/Actor_22/03-01-07-02-01-01-22.wav', '/content/ravdess/Actor_22/03-01-07-01-01-02-22.wav', '/content/ravdess/Actor_22/03-01-07-01-01-01-22.wav', '/content/ravdess/Actor_22/03-01-07-02-02-02-22.wav', '/content/ravdess/Actor_22/03-01-07-01-02-01-22.wav', '/content/ravdess/Actor_22/03-01-07-01-02-02-22.wav', '/content/ravdess/Actor_22/03-01-07-02-02-01-22.wav', '/content/ravdess/Actor_02/03-01-07-02-02-02-02.wav', '/content/ravdess/Actor_02/03-01-07-01-01-01-02.wav', '/content/ravdess/Actor_02/03-01-07-01-02-01-02.wav', '/content/ravdess/Actor_02/03-01-07-01-01-02-02.wav', '/content/ravdess/Actor_02/03-01-07-01-02-02-02.wav', '/content/ravdess/Actor_02/03-01-07-02-01-02-02.wav', '/content/ravdess/Actor_02/03-01-07-02-01-01-02.wav', '/content/ravdess/Actor_02/03-01-07-02-02-01-02.wav', '/content/ravdess/Actor_11/03-01-07-01-01-01-11.wav', '/content/ravdess/Actor_11/03-01-07-01-02-02-11.wav', '/content/ravdess/Actor_11/

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt


In [ ]:
#dataset_array=np.array(dataset_array)
sr=16000 # standardly used in audio preprocessing, CDs have audio 44050 and using Nyquist frequency it should be more than half
signal_store_array = [[] for _ in range(6)] #we'll load files into this

In [ ]:
print("anger:" , len(dataset_array[0]))
print("disgust:" , len(dataset_array[1]))
print("happy:" , len(dataset_array[2]))
print("neutral:" , len(dataset_array[3]))
print("sad:" , len(dataset_array[4]))
print("fear:" , len(dataset_array[5]))



anger: 1923
disgust: 1923
happy: 1923
neutral: 1895
sad: 1923
fear: 1923


Cool! now we'll see one or two waveforms and listen to them to ensure sab sahi hai

note that at this point the signal_store_array is a 3d list, and its [i][j][0]th element has an array that consists of all signal points:

The length of each is variable

This shouldn't be a problem once I extract the features, I guess, unless the looping I use requires same time period, whoch shouldn't be the case, I guess only sr will be needed, if that happens I need to do something - maybe look at each dataset and truncate off the rest portions

now I have to select which features to load, and save them in a dataframe, input will go as the [i][j][0] which is the array containing datapoints, I hope the variable length doesn't cause an issue

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
! pip install accelerate -U

In [ ]:
! pip install datasets evaluate soundfile librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
import pandas as pd
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments, Wav2Vec2Processor
from sklearn.model_selection import train_test_split

class EmotionDataset(Dataset):
    def __init__(self, csv_file, processor, target_sampling_rate=16000):
        self.data = pd.read_csv(csv_file)
        print(self.data.columns)  # Debug: Print columns to verify
        self.file_paths = self.data['file_path'].values
        self.labels = self.data['label'].values
        self.processor = processor
        self.target_sampling_rate = target_sampling_rate
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]

        try:
            waveform, sample_rate = torchaudio.load(file_path)
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")
            return None

        if sample_rate != self.target_sampling_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sampling_rate)
            waveform = resampler(waveform)

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0)  # Convert to mono

        inputs = self.processor(waveform.numpy(), sampling_rate=self.target_sampling_rate, return_tensors="pt")
        return {
            'input_values': inputs.input_values.squeeze(0),  # Assumes single channel audio
            'labels': torch.tensor(label, dtype=torch.long)
        }

from transformers import DataCollatorWithPadding

class DataCollatorForWav2Vec2(DataCollatorWithPadding):
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        input_values = [feature["input_values"] for feature in features]
        labels = [feature["labels"] for feature in features]
        batch = self.processor.pad({"input_values": input_values}, padding=True, return_tensors="pt")
        batch["labels"] = torch.tensor(labels, dtype=torch.long)
        return batch


In [ ]:
from transformers import AutoProcessor,  AutoFeatureExtractor

processor =  AutoFeatureExtractor.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition" )

dataset = EmotionDataset(csv_file='dataset.csv', processor=processor)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Index(['file_path', 'label'], dtype='object')


In [ ]:
! pip install peft # abhi abhi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.6 MB/s eta 0:00:00


In [ ]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model #abhi abhi

In [ ]:
train_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = (preds == labels).mean()
    return {'accuracy': acc}

In [ ]:
from transformers import AutoModelForAudioClassification
model = AutoModelForAudioClassification.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition" , num_labels=len(set(dataset.labels)) ) #device map abhi abhi


config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-e

In [ ]:
#model.train() # model in training mode (dropout modules are activated)
#model = prepare_model_for_kbit_training(model) #abhi abhi both

NameError: name 'prepare_model_for_kbit_training' is not defined

In [ ]:
'''if hasattr(model, "is_gradient_checkpointing") and model.is_gradient_checkpointing:
    # Enable gradient checkpointing
    model = model._prepare_model_for_gradient_checkpointing(model)'''

AttributeError: 'Wav2Vec2ForSequenceClassification' object has no attribute '_prepare_model_for_gradient_checkpointing'

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config) #abhi abhi both

NotImplementedError: 

In [ ]:
! pip install loralib

In [ ]:
import loralib as lora
model=lora.LoraModel(model)

AttributeError: module 'loralib' has no attribute 'LoraModel'

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    warmup_steps=2, #abhi abhi
    num_train_epochs=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorForWav2Vec2(processor=processor)
)

In [ ]:
model=trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss
